# Running our Models
Raveesha Gupta
Sarita Sapkota
Max Silverstein
December 14, 2017

Run this notebook after merging the data.

In [1]:
! conda install geopandas -yq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
from shapely.geometry import Point, Polygon

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split

In [ ]:
#Change Directory
import os
os.chdir('Project Files')

In [65]:
#Read data in
data=pd.read_csv('mergeddata.csv', header = 0, encoding = 'latin-1')


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
#Imputation: First impute at state level, then national level, all within the same year

data[['POPESTIMATE','density','Cardiovascular deaths', 'Medicare enrollees ', \
      'Divorced; Estimate; Population 15 years and over', 'credit_access', 'unemp_rate']] = data.groupby(\
      ['year', 'state'])['POPESTIMATE','density','Cardiovascular deaths', 'Medicare enrollees ', \
      'Divorced; Estimate; Population 15 years and over', 'credit_access', 'unemp_rate'].transform(lambda x: x.fillna(x.mean()))
data[['POPESTIMATE','density','Cardiovascular deaths', 'Medicare enrollees ', \
    'Divorced; Estimate; Population 15 years and over', 'credit_access', 'unemp_rate']] = data.groupby(\
     ['year'])['POPESTIMATE','density','Cardiovascular deaths', 'Medicare enrollees ', \
     'Divorced; Estimate; Population 15 years and over', 'credit_access', 'unemp_rate'].transform(lambda x: x.fillna(x.mean()))
    


In [67]:
#Check for null data
data.isnull().sum(axis=0)

Unnamed: 0                                                         0
CTYNAME                                                            0
year                                                               0
totalfilings                                                       0
state                                                              0
density                                                            0
POPESTIMATE                                                        0
credit_access                                                   6217
totpop                                                         27046
unemp                                                          27046
unemp_rate                                                         0
State                                                           4403
Cardiovascular deaths                                           3071
Unnamed: 4                                                     33100
Divorced; Estimate; Population 15 

In [68]:
#Set x and y vars
xvars=['density','Cardiovascular deaths', 'Medicare enrollees ', 'Divorced; Estimate; Population 15 years and over', \
       'credit_access', 'unemp_rate', 'POPESTIMATE' ]
#xvars=['density','Cardiovascular deaths', 'Medicare enrollees ', 'Divorced; Estimate; Population 15 years and over']
yvar=['totalfilings'] 
myvars = xvars + yvar
data[myvars].dropna().head()

,density,Cardiovascular deaths,Medicare enrollees,Divorced; Estimate; Population 15 years and over,credit_access,unemp_rate,POPESTIMATE,totalfilings
6217,83.619981,657.3,853.0,11.72381,73.687540,0.197931,49835.0,194.0
6218,108.278886,553.8,4217.0,10.80000,74.223203,0.244499,172851.0,670.0
6219,33.603797,601.9,620.0,11.72381,61.111111,0.197931,29736.0,146.0
6220,34.364316,684.8,360.0,11.72381,56.521739,0.197931,21410.0,105.0
6221,88.079121,658.6,758.0,11.72381,72.299652,0.197931,56863.0,187.0


In [69]:
#Train and Test Indices

dfdna = data[myvars].dropna().reset_index(drop=True)

df_train, df_test = train_test_split (dfdna[myvars], test_size=0.25, random_state=42)


In [92]:
#Cross Validation: Individual Models

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

names = ["LinearRegression", 'Ridge', 'Lasso']

regressors = [LinearRegression(), Ridge(), Lasso()]
for name, regressor in zip(names, regressors):
    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    for train_index, test_index in cv.split(df_train):
        X_train = df_train.loc[train_index][xvars].dropna()
        y_train = df_train.loc[train_index][yvar].dropna()
        X_test = df_train.loc[test_index][xvars].dropna()
        y_test = df_train.loc[test_index][yvar].dropna()
        regressor.fit(X_train, y_train)
        predicted = regressor.predict(X_test)
        score = regressor.score(X_test, y_test)
        print(name + ' R^2: '+str(100 * round(score, 3)))
        #print(name + ' MSE: '+ str(mean_squared_error(y_test,predicted)))
        print(name + ' MAE: '+ str(mean_absolute_error(y_test,predicted)))
        #for var, feat in zip(xvars, regressor.coef_[0]) :
        #   print(var + ': ' + str(100 * round(feat, 3)))
        

LinearRegression R^2: 60.5
LinearRegression MAE: 338.562281946
density: -7.8
Cardiovascular deaths: 66.6
Medicare enrollees : 9.2
Divorced; Estimate; Population 15 years and over: 1557.9
credit_access: 513.1
unemp_rate: -28490.3
POPESTIMATE: 0.1
LinearRegression R^2: 73.4
LinearRegression MAE: 294.266272875
density: -8.3
Cardiovascular deaths: 73.3
Medicare enrollees : 8.6
Divorced; Estimate; Population 15 years and over: 1170.9
credit_access: 572.4
unemp_rate: -24116.6
POPESTIMATE: 0.1
LinearRegression R^2: 45.7
LinearRegression MAE: 322.155901384
density: -8.7
Cardiovascular deaths: 73.6
Medicare enrollees : 8.4
Divorced; Estimate; Population 15 years and over: 913.3
credit_access: 634.3
unemp_rate: -21809.3
POPESTIMATE: 0.1
LinearRegression R^2: 71.0
LinearRegression MAE: 265.128327168
density: -9.6
Cardiovascular deaths: 71.0
Medicare enrollees : 8.3
Divorced; Estimate; Population 15 years and over: 1242.3
credit_access: 621.8
unemp_rate: -19610.4
POPESTIMATE: 0.1
LinearRegression 

TypeError: zip argument #2 must support iteration

In [84]:
#Cross Validation: Ensemble Models

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

names = ["RandomForestRegressor", 'GradientBoostingRegressor']

regressors = [RandomForestRegressor(), GradientBoostingRegressor()]

for name, regressor in zip(names, regressors):
    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    for train_index, test_index in cv.split(df_train):
        X_train = df_train.loc[train_index][xvars].dropna()
        y_train = df_train.loc[train_index][yvar].dropna()
        X_test = df_train.loc[test_index][xvars].dropna()
        y_test = df_train.loc[test_index][yvar].dropna()
        regressor.fit(X_train, y_train)
        predicted = regressor.predict(X_test)
        score = regressor.score(X_test, y_test)
        print(name + ' R^2: '+str(100 * round(score, 3)))
        #print(name +' MSE: '+ str(round(mean_squared_error(y_test,predicted),3)))
        print(name +' MAE: '+ str(round(mean_absolute_error(y_test,predicted), 3)))
        print("Feature Importance:")
        for var, feat in zip(xvars, regressor.feature_importances_) :
            print(var + ': ' + str(100 * round(feat, 3)))



/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor R^2: 88.5
RandomForestRegressor MAE: 112.91
Feature Importance:
density: 2.0
Cardiovascular deaths: 2.2
Medicare enrollees : 56.9
Divorced; Estimate; Population 15 years and over: 3.7
credit_access: 2.4
unemp_rate: 1.9
POPESTIMATE: 30.9


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor R^2: 86.3
RandomForestRegressor MAE: 115.922
Feature Importance:
density: 1.9
Cardiovascular deaths: 1.8
Medicare enrollees : 37.8
Divorced; Estimate; Population 15 years and over: 5.5
credit_access: 2.0
unemp_rate: 2.7
POPESTIMATE: 48.3


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


...

RandomForestRegressor R^2: 96.1
RandomForestRegressor MAE: 84.96
Feature Importance:
density: 2.5
Cardiovascular deaths: 1.9
Medicare enrollees : 35.9
Divorced; Estimate; Population 15 years and over: 6.1
credit_access: 1.6
unemp_rate: 2.9
POPESTIMATE: 49.0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor R^2: 94.9
RandomForestRegressor MAE: 113.824
Feature Importance:
density: 3.9
Cardiovascular deaths: 1.9
Medicare enrollees : 31.4
Divorced; Estimate; Population 15 years and over: 6.4
credit_access: 2.0
unemp_rate: 4.7
POPESTIMATE: 49.7


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor R^2: 91.2
GradientBoostingRegressor MAE: 147.387
Feature Importance:
density: 13.9
Cardiovascular deaths: 5.1
Medicare enrollees : 15.2
Divorced; Estimate; Population 15 years and over: 6.9
credit_access: 6.9
unemp_rate: 6.7
POPESTIMATE: 45.3
GradientBoostingRegressor R^2: 92.5
GradientBoostingRegressor MAE: 142.944
Feature Importance:
density: 14.6
Cardiovascular deaths: 3.4
Medicare enrollees : 15.4
Divorced; Estimate; Population 15 years and over: 5.9
credit_access: 5.7
unemp_rate: 9.5
POPESTIMATE: 45.4
GradientBoostingRegressor R^2: 78.2
GradientBoostingRegressor MAE: 155.158
Feature Importance:
density: 11.9
Cardiovascular deaths: 8.0
Medicare enrollees : 17.2
Divorced; Estimate; Population 15 years and over: 8.5
credit_access: 5.6
unemp_rate: 6.4
POPESTIMATE: 42.4
GradientBoostingRegressor R^2: 93.9
GradientBoostingRegressor MAE: 121.417
Feature Importance:
density: 15.9
Cardiovascular deaths: 8.2
Medicare enrollees : 13.3
Divorced; Estimate; Population 

In [83]:
#Cross Validation by Year

data['year'] = data['year'].astype('int64')
data = data[(data.year != 2005)]
data = data[(data.year != 2006)]
years = data['year'].unique()

for year in range(2008,2014):
    test = data[(data['year']==year)]
    #train = data[(data['year'] < year)]
    train = data[(data.year == year-3) | (data.year == year-2) |(data.year ==  year-1)]
    for name, regressor in zip(names, regressors):
        X_train = train[xvars].dropna()
        y_train = train[yvar].dropna()
        X_test = test[xvars].dropna()
        y_test = test[yvar].dropna()
        regressor.fit(X_train, y_train)
        predicted = regressor.predict(X_test)
        score = regressor.score(X_test, y_test)
        print(name + ' R^2 '+ ' for Year '+str(year)+': ' + str(100 * round(score, 3)))
        print(name + ' MAE '+ ' for Year '+str(year)+': '+ str(round(mean_absolute_error(y_test,predicted),3)))
        for var, feat in zip(xvars, regressor.feature_importances_):
            print(var + ': ' + str(100 * round(feat, 3)))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomForestRegressor R^2  for Year 2008: 70.2
RandomForestRegressor MAE  for Year 2008: 129.043
density: 5.2
Cardiovascular deaths: 7.9
Medicare enrollees : 24.8
Divorced; Estimate; Population 15 years and over: 4.2
credit_access: 0.9
unemp_rate: 1.2
POPESTIMATE: 55.8
GradientBoostingRegressor R^2  for Year 2008: 74.4
GradientBoostingRegressor MAE  for Year 2008: 135.735
density: 22.7
Cardiovascular deaths: 14.2
Medicare enrollees : 29.6
Divorced; Estimate; Population 15 years and over: 8.5
credit_access: 2.7
unemp_rate: 1.3
POPESTIMATE: 21.1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor R^2  for Year 2009: 74.2
RandomForestRegressor MAE  for Year 2009: 172.469
density: 2.9
Cardiovascular deaths: 7.1
Medicare enrollees : 40.9
Divorced; Estimate; Population 15 years and over: 3.2
credit_access: 0.9
unemp_rate: 2.6
POPESTIMATE: 42.4


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

GradientBoostingRegressor R^2  for Year 2012: 91.1
GradientBoostingRegressor MAE  for Year 2012: 152.851
density: 14.8
Cardiovascular deaths: 6.6
Medicare enrollees : 17.0
Divorced; Estimate; Population 15 years and over: 7.5
credit_access: 6.2
unemp_rate: 8.8
POPESTIMATE: 39.2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor R^2  for Year 2013: 85.4
RandomForestRegressor MAE  for Year 2013: 117.852
density: 3.1
Cardiovascular deaths: 1.5
Medicare enrollees : 52.7
Divorced; Estimate; Population 15 years and over: 2.4
credit_access: 0.8
unemp_rate: 4.0
POPESTIMATE: 35.5


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor R^2  for Year 2013: 84.6
GradientBoostingRegressor MAE  for Year 2013: 151.141
density: 14.9
Cardiovascular deaths: 5.3
Medicare enrollees : 25.7
Divorced; Estimate; Population 15 years and over: 7.3
credit_access: 5.9
unemp_rate: 9.6
POPESTIMATE: 31.4
